# Финансовая рекомендательная система (CatBoost)

In [49]:

import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [50]:

# Загрузка данных
users = pd.read_csv("users.csv")
products = pd.read_csv("products.csv")
interactions = pd.read_csv("interactions.csv")


In [51]:
print(f'Форма таблицы users: {users.shape}')
users.head(3)

Форма таблицы users: (100, 5)


,user_id,age,income,has_mortgage,goal
0,1,58,194231,1,пенсия
1,2,48,43986,1,покупка жилья
2,3,34,91858,1,пенсия


In [52]:
print(f'Форма таблицы products: {products.shape}')
products.head(3)

Форма таблицы products: (4, 5)


,product_id,type,min_income,goal,risk_score
0,101,вклад,40000,накопление,1
1,102,пенсионная программа,100000,пенсия,2
2,103,ипотека,50000,покупка жилья,3


In [53]:
print(f'Форма таблицы interactions: {interactions.shape}')
interactions.head(3)

Форма таблицы interactions: (324, 3)


,user_id,product_id,target
0,1,101,0
1,1,102,0
2,1,103,0


In [54]:
# Объединение users + products + interactions
data = interactions.merge(users, on="user_id").merge(products, on="product_id")
print(f'Форма итоговой таблицы data: {data.shape}')
data.head()

Форма итоговой таблицы data: (324, 11)


,user_id,product_id,target,age,income,has_mortgage,goal_x,type,min_income,goal_y,risk_score
0,1,101,0,58,194231,1,пенсия,вклад,40000,накопление,1
1,1,102,0,58,194231,1,пенсия,пенсионная программа,100000,пенсия,2
2,1,103,0,58,194231,1,пенсия,ипотека,50000,покупка жилья,3
3,1,104,0,58,194231,1,пенсия,инвестиции,80000,накопление,4
4,2,101,0,48,43986,1,покупка жилья,вклад,40000,накопление,1


In [55]:
# Выделение признаков и целевой переменной
X = data.drop(columns=["user_id", "product_id", "target"])
y = data["target"]

# Категориальные признаки
categorical_features = ["goal_x", "goal_y", "type"]
X[categorical_features] = X[categorical_features].astype("category")

In [56]:

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели CatBoost
model = CatBoostClassifier(verbose=100, cat_features=categorical_features)
model.fit(X_train, y_train)


Learning rate set to 0.005786
0:	learn: 0.6905435	total: 4.13ms	remaining: 4.13s
100:	learn: 0.5576618	total: 193ms	remaining: 1.72s
200:	learn: 0.4883721	total: 388ms	remaining: 1.54s
300:	learn: 0.4431886	total: 641ms	remaining: 1.49s
400:	learn: 0.4136671	total: 871ms	remaining: 1.3s
500:	learn: 0.3940742	total: 1.1s	remaining: 1.09s
600:	learn: 0.3727400	total: 1.33s	remaining: 886ms
700:	learn: 0.3542466	total: 1.57s	remaining: 669ms
800:	learn: 0.3399355	total: 1.82s	remaining: 451ms
900:	learn: 0.3263653	total: 2.04s	remaining: 224ms
999:	learn: 0.3090397	total: 2.29s	remaining: 0us


In [57]:

# Предсказание
y_pred = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC AUC:", roc_auc)


ROC AUC: 0.7997448979591837


In [58]:

# Таблица ранжирования
ids = data.loc[X_test.index, ["user_id", "product_id"]].copy()
ids["score"] = y_pred
ranked = ids.sort_values(by=["user_id", "score"], ascending=[True, False])
ranked.head()


,user_id,product_id,score
3,1,104,0.077809
7,3,104,0.111576
5,3,101,0.104224
9,4,102,0.050926
16,7,102,0.044117


In [62]:
model.save_model("catboost_model_ROC08.json", format="json")

In [60]:
products.head(3)

,product_id,type,min_income,goal,risk_score
0,101,вклад,40000,накопление,1
1,102,пенсионная программа,100000,пенсия,2
2,103,ипотека,50000,покупка жилья,3


In [34]:

# Сохраняем рекомендации
ranked.to_csv("recommendations.xls", index=False)
print("Файл recommendations.xls сохранён")


Файл recommendations.xls сохранён


### После 1-ого обучения получили RocAug 0.79, что не плохо, однако можно попробовать поработать с признаками и добавить несколько индикаторов.

In [35]:
# 1. Совпадение цели (goal_match)
data["goal_match"] = (data["goal_x"] == data["goal_y"]).astype(int)

In [36]:
# 2. Разница между доходом пользователя и минимальным доходом продукта
data["income_gap"] = data["income"] - data["min_income"]

In [37]:
# 3. Возрастная категория
data["age_group"] = pd.cut(data["age"], bins=[18, 30, 45, 60, 100], labels=["18-30", "31-45", "46-60", "60+"])

In [38]:
#4. Признак “риск подходит ли пользователю”
# Если у человека ипотека → низкий риск предпочтительнее

data["risk_sensitive"] = ((data["has_mortgage"] == 1) & (data["risk_score"] > 2)).astype(int)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   user_id         324 non-null    int64   
 1   product_id      324 non-null    int64   
 2   target          324 non-null    int64   
 3   age             324 non-null    int64   
 4   income          324 non-null    int64   
 5   has_mortgage    324 non-null    int64   
 6   goal_x          324 non-null    object  
 7   type            324 non-null    object  
 8   min_income      324 non-null    int64   
 9   goal_y          324 non-null    object  
 10  risk_score      324 non-null    int64   
 11  goal_match      324 non-null    int64   
 12  income_gap      324 non-null    int64   
 13  age_group       324 non-null    category
 14  risk_sensitive  324 non-null    int64   
dtypes: category(1), int64(11), object(3)
memory usage: 36.1+ KB


#### 1. goal_match
Совпадает ли цель пользователя с целью продукта.
Если пользователь хочет "накопление", а продукт предлагает "накопление" — шанс интереса выше.
#### 2. income_gap
Разница между доходом пользователя и минимальным доходом, требуемым для продукта.
Чем больше запас по доходу, тем выше вероятность, что продукт подходит.
#### 3. age_group
Категория возраста пользователя. Возрастные группы по-разному интересуются продуктами (пенсия интересует старших, ипотека — молодых).
18–30, 31–45, 46–60, 60+
#### 4. risk_sensitive
Флаг, показывающий, чувствителен ли пользователь к риску.
Если у пользователя есть ипотека, он, скорее всего, не будет интересоваться высокорискованными продуктами.


In [41]:
# Подготовка фичей
X = data[[
    "age", "income", "has_mortgage", "risk_score",
    "goal_match", "income_gap", "age_group", "type", "goal_x", "goal_y"
]]
y = data["target"]

In [44]:
# Кодирование категориальных признаков
X = pd.get_dummies(X)

# Разделение и обучение
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = CatBoostClassifier(verbose=100)
model.fit(X_train, y_train)

Learning rate set to 0.005786
0:	learn: 0.6896052	total: 1.99ms	remaining: 1.99s
100:	learn: 0.4775087	total: 132ms	remaining: 1.17s
200:	learn: 0.3936317	total: 263ms	remaining: 1.04s
300:	learn: 0.3463072	total: 396ms	remaining: 920ms
400:	learn: 0.3094400	total: 527ms	remaining: 787ms
500:	learn: 0.2803916	total: 657ms	remaining: 654ms
600:	learn: 0.2561185	total: 809ms	remaining: 537ms
700:	learn: 0.2347425	total: 943ms	remaining: 402ms
800:	learn: 0.2145625	total: 1.08s	remaining: 269ms
900:	learn: 0.1970488	total: 1.21s	remaining: 133ms
999:	learn: 0.1817602	total: 1.34s	remaining: 0us


In [46]:
# Предсказание и метрика
y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Ранжирование
ids = data.loc[X_test.index, ["user_id", "product_id"]].copy()
ids["score"] = y_pred_proba
ranked = ids.sort_values(by=["user_id", "score"], ascending=[True, False])
ranked.to_csv("recommendations_with_features.csv", index=False)

roc_auc

np.float64(0.7576530612244898)

Как можно заметить, дополнительные признаки не помогли, а напритив, ухудшили качество модели. Логичный вариант оставить 1ю версию модели с оценнкой 0,80

In [47]:
model.save_model("catboost_model.cbm")

In [48]:
products.to_csv("products.csv", index=False)